In [678]:
import numpy as np
from numpy.linalg import norm, inv, pinv
from fpylll import FPLLL, IntegerMatrix, SVP, LLL, BKZ, CVP
from fpylll.algorithms.bkz import BKZReduction
from scipy.special import gammainc, gammaincinv, gammainccinv
from copy import copy
import math
from multiprocessing import Pool, cpu_count
from svp import __svp as __svp__

import time

# start = time.time()
# print("hello")
# end = time.time()
# print(end - start)

np.random.seed(1337)
FPLLL.set_random_seed(1337)

In [679]:
def inverse_transform_sampling(R, sigma, n):
    r = np.random.normal(R, sigma)
    direction = np.random.normal(0, 1, n)
    return r * direction

In [680]:
def lattice_vector(v, B):
    # return B@[int(x) for x in inv(B)@v]
    return B@np.array([int(x) for x in pinv(B)@v])

In [681]:
def __svp(B, n, R, sigma, num_samples):
	s, l = -1, 2**n*norm(B)
	T1, T2=0,0
	for _ in range(num_samples):
		t1=time.time()
		x = inverse_transform_sampling(R, sigma, n)
		t2=time.time()
		x=lattice_vector(x, B)
		t3=time.time()
		if(norm(x)>0 and norm(x)<l):
			s, l=x,norm(x)
		T1+=t2-t1
		T2+=t3-t2
	print(T1,T2)
	return s,l

In [682]:
def parallel_svp(B, n, R, sigma, num_samples, processes=None):
    if processes is None:
        processes = cpu_count()
    print(f'computing with {processes} processes.')
    args = [(B, n, R, sigma, num_samples//processes) for _ in range(processes)]
    with Pool(processes) as pool:
        results = pool.starmap(__svp__, args)
    best_s, best_l = min(results, key=lambda x: x[1])

    return best_s, best_l

In [683]:
def __SVP(B, n, t):
	l, r = 0, min(norm(x) for x in B)
	s, L = -1, r
	print(l,r)
	for _ in range(10):
		m=(l+r)/2
		print(_, int(m), int(L), '->', int(l), int(r))
		_s, _L=parallel_svp(B, n, m, m, t)
		if(_L<L):
			s, L=_s, _L
		else:
			_s, _L = s, L
		r=_L
		if(_L>m):
			l=m
		else:
			l=m/2
	return s,L

In [684]:
def generate_random_instance(b, n):
	A = IntegerMatrix(n,n)
	A.randomize("uniform", bits=b)
	# A=[[-1248884424688331397, 217379346648022931, 968695965772720018],
	#    [-252808953422802017, 44003627585902284, 196091014068821624],
	#    [-480021271462343831, 83551935074919420, 372328023281023077]]
	# A = IntegerMatrix.from_matrix(A)
	# print(A)
	return A

In [685]:
def generate_hard_instance(n, q, r):
	"""
	Generates a hard SVP instance with a short basis using Ajtai99.
    n: Lattice dimension
    q: Modulus for lattice entries (should be prime or appropriate for lattice properties)
	r: Number of initial random vectors
	"""
	# Step 1: Generate random vectors `u_i` in `Z_q^n`
	# Step 2: Construct the matrix `A` following Ajtai's block structure
	u_vectors = np.random.randint(0, q, (r, n))
	A1 = np.eye(r, dtype=int)
	A2 = np.random.randint(-q//2, q//2, (n - r, n - r))
	A = np.block([
        [A1, np.zeros((r, n - r), dtype=int)], 
        [np.zeros((n - r, r), dtype=int), A2]
    ])
    
    # Step 3: Define `B` as a lower triangular matrix with padding for proper dimensions
    # Step 4: Construct lattice basis using matrix multiplication with modulo q
	B_core = np.tril(np.random.randint(-r, r, (n - r, n - r)))
	B = np.hstack([np.zeros((n - r, r), dtype=int), B_core])  # Pad B to shape (n - r, n)
	lattice_basis = (A @ np.vstack([u_vectors, B]) % q)

    # Step 5: Enforce norm constraints on basis vectors
	norm_bound = np.sqrt(n) * n
	for i in range(lattice_basis.shape[0]):
		vector_norm = np.linalg.norm(lattice_basis[i])
		if vector_norm > norm_bound:
			lattice_basis[i] = (lattice_basis[i] / vector_norm) * norm_bound

	return lattice_basis.astype(int)
# generate_hard_instance(10,97,3)

In [686]:
def reduced_basis(X, n):
	B=list([])
	for i in range(n):
		B.append([0 for _ in range(n)])
		for j in range(n):
			B[i][j]=int(X[i][j])
	A = IntegerMatrix.from_matrix(B)

	barA=LLL.reduction(A)
	# A = BKZ.reduction(A, BKZ.Param(20))
	
	B=list([])
	for i in range(n):
		B.append([0 for _ in range(n)])
		for j in range(n):
			B[i][j]=int(barA[i][j])
	B=np.array(B).T
	return A, B

In [687]:
n, b= 12, 32
X = generate_random_instance(b, n)

# n, p, r= 17, 97, 3
# X = generate_hard_instance(n, p, r)

A, B = reduced_basis(X, n)

X=copy(A)
SVP.shortest_vector(A)
s, l = A[0], norm(A[0])
print('------------------------------------')
print('fpylll solution:')
print([int(x) for x in s],'\n Norm:', l)
# print([int(x) for x in inv(B)@s])

print('------------------------------------')
print('my solution:')
C = 1															#Working fine, but C~20: 2*e*pi
num_samples=int(2**(C*n))*int(math.log2(n))
# s,_l=__svp(B, n, l, l, num_samples)
# s, _l=__SVP(B, n, num_samples)
s, _l = parallel_svp(B, n, l, l, num_samples)

print([-int(x) for x in s],'\n Norm:', _l)

print('------------------------------------')
print('verify solution:')
# print(X)
# print('coefficients vector:  ',[int(x) for x in pinv(B)@s])
# print('reconstructed vector: ' ,list(B@[int(x) for x in pinv(B)@s]))

v0 = CVP.closest_vector(X, tuple([int(x) for x in s]))
# print('Verify with CVP:      ',[int(x) for x in v0],'\n norm:',norm(v0))
print('Verdict:', bool(abs(norm(v0)-_l)<1e-3), ', Comparing solutions:', bool(l<=_l+1e-6))

------------------------------------
fpylll solution:
[-452848946, -1149864540, -439679276, 499762204, 856713830, -1398252645, -838336197, -1077290477, 1735376101, 638539914, 643434089, -605019993] 
 Norm: 3274460597.3872533
------------------------------------
my solution:
computing with 8 processes.
[-452848946, -1149864540, -439679276, 499762204, 856713830, -1398252645, -838336197, -1077290477, 1735376101, 638539914, 643434089, -605019993] 
 Norm: 3274460597.3872533
------------------------------------
verify solution:
Verdict: True , Comparing solutions: True
